# Orquestador de ventas 360: resumen

## Input
1. Excel de demanda desagregada: con esta información se genera lista de claves y máximos. 
## Output
1. PDF's con nombre estructurado en folder específico. 
2. Base de datos con información de los contratos y convenios modificatorios generados con el string. 

## Notas
Diseño esto para, partiendo de la demanda desagregada: 
- Capturar contratos
- Descargar xlsx de sistemas SAI, CAMUNDA, SAGI, ZOHO y PISP. 
- Consultar exceles internos
- Organizar las órdenes descargadas
- Generar los pickle de las órdenes
Todo lo anterior es la información mínima indispensable para poder generar información de ventas, que será la referencia interna sobre la que desplegaremos el alimentado de reportes y la generación de reglas para dar retro al usuario.

In [ ]:
# Importa librerías y carpetas con funciones
import sys
import os
import yaml
import pandas as pd
import glob
# Define the root and script directory
folder_root = os.getcwd()  # Get current directory (where Orquestación.ipynb is)
script_folder = os.path.join(folder_root, "Scripts")  # Path to 'Scripts'
# Ensure the script folder is added to sys.path
if script_folder not in sys.path:
    sys.path.append(script_folder)
contratos_library_scripts = os.path.join(script_folder, "Libreria_contratos")
if contratos_library_scripts not in sys.path:
    sys.path.append(contratos_library_scripts)

sanciones_folder = os.path.join(script_folder, "Libreria_SancionesIMSSB")
if sanciones_folder not in sys.path:
    sys.path.append(sanciones_folder)

In [ ]:
# Cargar librerías internas
from folders_files_open import create_directory_if_not_exists
from STEP_C_PDFhandling import STEP_C_read_PDF_from_source
from dataframes_generation import create_dataframe #(extension, dataframe_name, columns, output_folder)
# Generador de carpetas
from folders_files_open import create_directory_if_not_exists
# Administración de contratos:
from administracion_de_contratos import administracion_de_contratos
from sanciones_IMSSB_processing import print_columns

In [ ]:
#Cargar los folders requeridos. 
working_folder = desagregadas_folder = os.path.join(folder_root, "Implementación")
desagregadas_folder = os.path.join(folder_root, "Implementación", "Desagregadas")
create_directory_if_not_exists(working_folder)
create_directory_if_not_exists(desagregadas_folder)

# Procesa inputs
Las siguiente será la estructura de la carpeta con la información.

### (1) Carga y (2) limpia demanda desagregada
En caso de que no tengas aún demanda desagregada por procedimiento ejecuta los siguientes pasos. 
El objetivo es generar un dataframe estandarizado con Institución, piezas, procedimiento, y precio. 

In [ ]:
# Seleccionar el archivo XLSX de la carpeta para procesar.
XLSX_file_list = glob.glob(os.path.join(desagregadas_folder, "*.xlsx"))
XLSX_file_list.sort()
print(f"Found {len(XLSX_file_list)} files:")
for f in XLSX_file_list:
    print(f"\t {os.path.basename(f)}")

# prompt until a valid filename is entered
while True:
    XLSX_input = input("Ingresa el excel de la demanda desagregada:")
    # compare against basenames
    basenames = [os.path.basename(f) for f in XLSX_file_list]
    if XLSX_input in basenames:
        selected_path = os.path.join(desagregadas_folder, XLSX_input)
        # Using os.path
        parts    = os.path.normpath(selected_path).split(os.sep)
        folder1, folder2, fname = parts[-3], parts[-2], parts[-1]
        print(f"✅ Archivo seleccionado: {folder1}/{folder2}/{fname}")
        break
    else:
        print("❌ Selección no válida, elije un archivo de la lista. ")


In [ ]:
# Selecciona la hoja e imprime las columnas
# 1. load the workbook and list sheets
xls = pd.ExcelFile(selected_path)
sheets = xls.sheet_names
print("Available sheets:")
for name in sheets:
    print(" •", name)

# 2. prompt until a valid sheet is chosen
while True:
    selected_sheet = input("Select a sheet by name: ")
    if selected_sheet in sheets:
        break
    print("❌ Invalid sheet. Please choose one from the list above.")

# 3. read that sheet and print file path + its columns
df_input_xlsx = pd.read_excel(selected_path, sheet_name=selected_sheet)
print(f"✅ {XLSX_input} — Columnas:", )
print_columns(list(df_input_xlsx.columns))

In [ ]:
# Genera un dataframe con columnas adecuadas
from PROCESA_DESAGREGADA_input import standarized_dataframe_generation
standarized_base_columns = ["Institución", "Procedimiento", "Clave", "Descripción", "Precio", "Piezas"]
raw_input_excel_columns = list(df_input_xlsx.columns)
#print_columns(list(raw_input_excel_columns.columns), n_cols=3)

df_clean = standarized_dataframe_generation(df_input_xlsx, standarized_base_columns)


### (3) Transformar demanda desagregada
Con el dataframe adecuadamente formateado, podemos hacer la agrupación por institución y clave, si se confirma que es lo esperado se exporta a pickle. 

In [ ]:
# Agregamos por institución y procedemos a guardar el pickle. 
from desagregada_to_pickle import save_to_pickle
#print(df_clean.columns)
#print(df_clean.info())
df_grouped = (
    df_clean
    .groupby(['Institución','Clave'], as_index=False)
    .agg(
        Piezas        = ('Piezas',       'sum'),
        Procedimiento = ('Procedimiento','first'),
        Descripción   = ('Descripción',  'first'),
        Precio        = ('Precio',       'first')
    )
)
#print(df_grouped.columns)
#print(df_grouped.info())

df_grouped.head(20)
#print(df_grouped['Procedimiento'].unique())

while True:
    user_revision = input("¿El dataframe es adecuado y está listo para ser guardado? (si/no): ")
    respuesta = user_revision.strip().lower()

    if respuesta == 'si':
        save_to_pickle(df_grouped, desagregadas_folder)
        break

    elif respuesta == 'no':
        print("Repite los pasos hasta que estés seguro.")

    else:
        print("Respuesta no válida. Por favor, responde 'si' o 'no'.")

# Administrador de contratos
Aquí empieza todo, la demanda desagregada nos da una lista de instituciones, sus piezas máximas y mínimas

## Elije el procedimiento
En esta sección definimos a qué procedimiento corresponde el contrato que vamos a cargar. 

In [ ]:
# Carga la demanda desagregada que aplique
from desagregada_to_pickle import load_pickles

df_proccedure, procedimiento = load_pickles(desagregadas_folder)
df_proccedure.info()
df_proccedure.sample(4)


## Capturar contrato - Exportar base de contratos
1) Captura de contrato: te va haciendo preguntas hasta que tienes un PDF con la leyenda que necesitamos en su cuerpo.
2) Extrae la información de los diccionarios del PDF y genera un archivo .pickle con la información.
3) Exporta pickle a excel para consulta de los usuarios.

In [ ]:
from folders_files_open import create_directory_if_not_exists
from desagregada_to_pickle import base_procedimiento_pickle
Columnas = ['Institución', 'Procedimiento', 'Contrato', 'Fecha Inicio', 'Fecha Fin', 'Productos y precio', 'Total', 'Nombre del archivo', 'Estatus', 'Convenio modificatorio', 'Objeto del convenio']
print(procedimiento)
Folder_procedimiento = os.path.join(folder_root, "Implementación", "Contratos", f"{procedimiento}")
create_directory_if_not_exists(Folder_procedimiento)
base_procedimiento = base_procedimiento_pickle(Folder_procedimiento, procedimiento, Columnas) 
print(f"\nprocedimiento:{procedimiento}")

# Capturar un nuevo contato
#administracion_de_contratos(data_warehouse, working_folder, libreria_contratos)

In [ ]:
from STEP_A_orchestration import STEP_A_orchestration
STEP_A_orchestration(df_proccedure, procedimiento, Folder_procedimiento)
print(base_procedimiento.head(1))
#STEP_C_read_PDF_from_source(libreria_contratos)